# Data cleaning and formatting

Here the data is cleaned and rationalized in order to cast it into a _pandas_ dataframe.
We focus mostly into filling numerical values from literal fields (_e.g._ the floor from the description).
**To mark an empty field, we use negative numbers.** Every numerical field has to be positive, and negative numbers are easy to filter out.

However, it will happen that we need more massaging for particular plots, there is no doubt.

In [6]:
import cPickle as pkl
import copy
dbfname = 'data/locations.pkl'
weekdays_en=['monday','tuesday','wednesday','thursday','friday','saturday','sunday']
paris_metro_stations = pkl.load(open('data/metroParis.pkl','rb'))
from fuzzywuzzy import process as fwprocess
import fuzzywuzzy as fuzwuz

In [7]:
db=pkl.load(open(dbfname,'rb'))
print '{0:d} locations in db'.format(len(db))

601 locations in db


In [8]:
Mlen = max(len(j) for j in db);
mlen =  min(len(j) for j in db);
# store the longer and the shorter for inspection
Mj = [j for j in db if len(j)==Mlen][0]
mj = [j for j in db if len(j)==mlen][0]
# Mj
print 'Max and min apartments records:', Mlen, mlen
if Mlen != mlen:
    print 'you need to adjust the lenghts, somehow'

Max and min apartments records: 22 13
you need to adjust the lenghts, somehow


In [9]:
# let refine the REGEXes
# These are very similar to the ones we used to parse the html
# I improved re_rue, and re_arr2
import re
re_rue = re.compile(r'(?:rue|avenue|boulevard|bd|place) .*?[\d,.]+[^.,]*?',re.IGNORECASE)
re_arr1 = re.compile(r'(Paris \d{1,2}\s*(?:th|st|nd|rd){0,1})',re.IGNORECASE)
re_arr2 = re.compile(r'(\d{1,2}\s*(?:th|st|nd|rd){0,1}\s{0,2}arr)',re.IGNORECASE|re.MULTILINE)
re_metro = re.compile(r'Metro\s*(?:station|line)\s*.+?[\s|$]',re.IGNORECASE|re.MULTILINE)
re_floor = re.compile(r'(?:ground|\d+|\d+[a-z]{2,2})\s*floor',re.IGNORECASE|re.MULTILINE)

get_floor = lambda x: ', '.join(re_floor.findall(x))
get_metro = lambda x: ', '.join(re_metro.findall(x))

def extloc(text):
    _=''
    for rg in (re_rue,re_arr1,re_arr2):
        out = rg.findall(text)
        _+=', '.join(out)
    return _

extloc(Mj[-1])

u'13th arr'

In [10]:
# shorten record removing useless info about meeting days
#  -> we discovered this by printing Mj
# everything else should be kept

def wkinstr(x):
    for wk in weekdays_en:
        if wk in x:
            return True
    return False

db2=copy.deepcopy(db)
for j,row  in enumerate(db2):
    db2[j]=[feature for feature in row if not wkinstr(feature)]
                
Mlen2 = max(len(j) for j in db2);
mlen2 =  min(len(j) for j in db2);
# store the longer and the shorter for inspection
Mj2 = [j for j in db2 if len(j)==Mlen2][0]
mj2 = [j for j in db2 if len(j)==mlen2][0]

print 'Max and min apartments records:', Mlen2, mlen2
if Mlen2 != mlen2:
    print 'you need to adjust the lenghts, somehow'

Max and min apartments records: 19 12
you need to adjust the lenghts, somehow


## Description to Metro to District
The next two functions use fuzzy string-matching to infer 
the flat position from the metro stations matched in the text.

The list of metro stations is `paris_metro_stations = [('short name',arr,'long name','all arr'),...]`

The list is obtained from wikipedia, in [Paris Metro](Paris Metro.ipynb).

In [11]:
def contains(a,b):
    # returns true if any item of a is contained in some items of b (I hope English is correct)
    for j in a:
        if j in b: return True
    return False

def getmetros(text):
    # guess position from metro stations cited in text
    # returns the list [(fuzzy score, index of the paris_metro_stations list),..] 
    #                if fuzzy score bigger than 50
    # if a score is bigger than 99, stops and returns the one-element list [(fuzzy score, index),] 
    matches = []
    lowtext = text.lower()
    
    for idx,pms in enumerate(paris_metro_stations):
        shortname, arrondissement,longname,longarr = pms
        score = fuzwuz.fuzz.token_set_ratio(shortname.lower(),lowtext)
        if score > 99:
            return [(score,idx),]
        if score > 50:
            matches.append((score,idx))
    return matches

def getcleanmetros(row):
    # get best estimated metro nearby the flat from the flat description (row[-1])
    # ok. This is tedious:
    # 1) if no metros are found, return empty string
    # 2) if there is only one possible metro (eg. if there is only one match with score> 50%, or one score >99%)
    #    return the record. 
    # 
    # At this point the list 'out' contains multiple matches, we need to find out the best, and return it.
    # 3) if the current location field (row[6]) contains a number (eg, arrondissement)
    #    compares the arrondissment of the estimated train stations with the current estimated arrondissement, 
    #    and returns the best match
    # 3a) NOTE: some train staions are not identified by a numerical arrondissement. see code for details
    # 4) else: returns the metro with the highest score
    #
    # This function:
    # returns the paris_metro_station record of the best match
    # or an empty string
    out = getmetros(row[-1])
    
    if not len(out):
        return []
    
    if len(out) == 1:
        j = out[0]
        return paris_metro_stations[j[1]]
    
    if contains('123456789',row[6]):
        # then we can match some arrondissmenent number, otherwise returns best match
        results = []
        score_old = 0
        exception_triggered = False
        for j in out:
            arr = paris_metro_stations[j[1]][1]
            try:    
                arr /2 # test if number
                score =  fuzwuz.fuzz.token_set_ratio(arr,row[6])
                if score>score_old and score > 75:
                    score_old = score
                    results = paris_metro_stations[j[1]]
            except Exception:
                # in case we find nothing better, we will resort to a non numerical arrondissement :/
                exception_triggered = j
        if exception_triggered and not len(results):
            return paris_metro_stations[exception_triggered[1]]
        else:
            return results
    else:    
        score_old = 0
        results = []
        for j in out:
            score = j[0]
            if score > score_old:
                score_old = score
                results = paris_metro_stations[j[1]]
        return results
# test on a record we had
# getcleanmetros(Mj)

In [12]:
# Rewrite the new records, using the new regex to match the arrondissement
# and location and metro finder
# and the floor                                                             <-------- NOW
from string import atoi
import pandas as pd

def replace(a,b,c):
    #replace list a, with b (either const or list) in str(c)
    c = unicode(c)
    if len(b)>=len(a):
        for _out,_in in zip(a,b):
            c = c.replace(unicode(_in),unicode(_out))
    else: 
        _out = unicode(b)
        for _in in a:
            c = c.replace(unicode(_in),_out)
    return c.strip()

def correct_encoding_CL(text):
    # encoding of some french text is screwed up - this is a rough correction
    # the dictionary is handpicked from the text
    fr2en=[[u'aª','e'],
     [u'ÃƒÂ','a'],
     [u'a©','e'],
     [u'a¨','e'],
     [u'a®','i'],
     [u'Ãƒâ€°','E'],
     [u'Ã‚Â²','2'],
     [u'aª','e'],
     [u'a¹','u'],
     [u'§','c'],
     [u'\xa0',''],
     [u'Ã©','e'],
     [u'\xe8','e'],
     [u'\xe0','a']
     ]
    fr2en_b = [j[0] for j in fr2en]
    fr2en_a = [j[1] for j in fr2en]
    out = replace(fr2en_a,fr2en_b,c=text)
#     out.encode('ascii') # activate to check that we know enough replacements..
    return out

def work1(apa):
    apa=copy.deepcopy(apa)
    apa[6] = extloc(apa[-1])
    apa[6] = replace('.,/()[]{}!~?#@*&^%<>\\`"\'','',apa[6])
    metros = getcleanmetros(apa)
    # now clean the encoding -- dunno if it is good or bad to do it before the fuzzy matching
    for i,newfeat in enumerate(apa):
        apa[i]=correct_encoding_CL(newfeat)
    
    if not contains('123456789',apa[6]) and len(metros):
        apa[6] += ' ' + metros[-1] + ' arrondissement'
        apa[6].strip()
        
    if not len(metros):
        # we did not find any metro, but still we want these fields in the table
        metros=('',-1)
    apa[7] = metros[0]
    apa.insert(8,metros[1])
    if not isinstance(apa[3],int):
        apa[3]= atoi(apa[3][:-2])
    if not isinstance(apa[4],int):
        apa[4] = atoi(apa[4])
    if not isinstance(apa[1],int):
        apa[1] = atoi(apa[1])
    
    # floor
    re_floor = re.compile('[^0-9]')
    floor = apa[5]
    floor = floor.lower().replace('ground','0')
    floor = re_floor.sub('',floor) # strip text, leave numbers only
    if not floor: floor = -1
    else: floor = atoi(floor)
    apa[5] = floor
    return apa
work1(Mj2)
print 

## Make one table from different-lenght lists

In [38]:
unlisted_features = []
unfeat_slice=slice(11,-2)
for row in db2:
    for feature in row[unfeat_slice]:
        unlisted_features.append(feature)

unlisted_features_set=set(unlisted_features)
all_features_titles = ['ins date','idx','url','sqm','price','floor','loc','metro','district','date in','text in']
features_suffix = len(all_features_titles)
all_features_titles.extend(unlisted_features_set)
all_features_titles.extend(['title','desc'])
print all_features_titles

['ins date', 'idx', 'url', 'sqm', 'price', 'floor', 'loc', 'metro', 'district', 'date in', 'text in', u'flat', u'apartment', u'carport', u'furnished', u'duplex', u'no smoking', u'street parking', u'house', u'laundry in bldg', u'w/d hookups', u'dogs are OK - wooof', u'w/d in unit', u'no parking', u'townhouse', u'no laundry on site', u'laundry on site', u'attached garage', u'wheelchair accessible', u'condo', u'off-street parking', 'title', 'desc']


In [15]:
new_db = []
count = 0

for row in db2:
    apa = work1(row)
    features = apa[:features_suffix]
    features.extend([j in apa[unfeat_slice] for j in unlisted_features_set])
    features.extend(apa[-2:])
    new_db.append(features)
    
    count += 1
    print count,'\r',

## Save to hdf5 
We save to hdf5 that is easy enough to handle (and to corrupt, seriously).

We now have quite a bit of features per each flat. 
Some columns are clearly correlated: eg all the parking stuff.. but let's try to get something done nonetheless

In [33]:
import pandas as pd
dataframe = pd.DataFrame(data=new_db, columns=all_features_titles)
dataframe.to_hdf('data/CL_flats.h5','CL_flats')

In [31]:
cols=dataframe.columns
print cols

Index([             u'ins date',                   u'idx',
                         u'url',                   u'sqm',
                       u'price',                 u'floor',
                         u'loc',                 u'metro',
                    u'district',               u'date in',
                     u'text in',                  u'flat',
                   u'apartment',               u'carport',
                   u'furnished',                u'duplex',
                  u'no smoking',        u'street parking',
                       u'house',       u'laundry in bldg',
                 u'w/d hookups',   u'dogs are OK - wooof',
                 u'w/d in unit',            u'no parking',
                   u'townhouse',    u'no laundry on site',
             u'laundry on site',       u'attached garage',
       u'wheelchair accessible',                 u'condo',
          u'off-street parking',                 u'title',
                        u'desc'],
      dtype='object')


### A style

In [1]:
from IPython.display import HTML
import urllib2

style=open('style.css','r').read()
HTML("""
<style>{0}</style>
""".format(style))